# Pipelines

Aprenderemos a trabalhar *corretamente* com pipelines no scikit-learn.

É importante entender que o conceito de pipelines é válido para qualquer conjunto de biblitecas, linguagens e ferramentas.

Todo projeto precisa de um Pipeline de dados bem definido, bem documento e bem testado. Nesta sessão, abordaremos apenas o pipeline de transformação de dados para alimentar nossos modelos de ML.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import randint, uniform

# Carregando os dados
# https://archive.ics.uci.edu/dataset/19/car+evaluation
column_names = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
data = pd.read_csv("../dados/carros_usados/car.data", header=None, names=column_names)

### Dicionário de dados dataset

| Attribute Name | Role    | Type        | Demographic | Description                                | Units | Missing Values |
|----------------|---------|-------------|-------------|--------------------------------------------|-------|----------------|
| buying         | Feature | Categorical |             | buying price                               |       | false          |
| maint          | Feature | Categorical |             | price of the maintenance                   |       | false          |
| doors          | Feature | Categorical |             | number of doors                            |       | false          |
| persons        | Feature | Categorical |             | capacity in terms of persons to carry      |       | false          |
| lug_boot       | Feature | Categorical |             | the size of luggage boot                   |       | false          |
| safety         | Feature | Categorical |             | estimated safety of the car                |       | false          |
| class          | Target  | Categorical |             | evaluation level (unacceptable, acceptable, good, very good) | | false |


In [2]:
# Usaremos doors e persons como variaveis numericas neste exemplo
# Convertendo "5-more" e "more" para 5
data['doors'] = data['doors'].replace('5more', 5).astype(int)
data['persons'] = data['persons'].replace('more', 5).astype(int)

### Definição do conjunto de Treino e Validação 80% e Teste (20%)

In [3]:
# Dividindo os dados em conjuntos de treinamento (90%) e teste (10%)
# Definindo X e y
X = data.drop(columns=["class"])
y = data["class"]
X_temp, X_test_final, y_temp, y_test_final = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

### Definição do conjunto de Treino 80% e Validação 20%, já dentro dos 80% dos dados que não são de teste.

In [4]:
# Dividindo o conjunto de treinamento em subconjuntos de treinamento e validação para RandomizedSearchCV
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.8, random_state=42, stratify=y_temp)

### Definição dos pre-processamentos necessários para dados numéricos e categóricos

In [5]:
# Definindo colunas numéricas e categóricas
numeric_features = ['doors', 'persons']
categorical_features = ['buying', 'maint', 'lug_boot', 'safety']

# Criando o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OrdinalEncoder(), categorical_features)])

### Definição do Pipeline

In [6]:
# Criando o pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())  # Inicial com LogisticRegression
])

### Definição do Espaço de Busca de modelos e seus hiperparâmetros

In [7]:
# Hiperparâmetros e modelos
param_distributions = {
    'LogisticRegression': {
        'model__C': uniform(0.001, 10),
        'model__penalty': ['l1', 'l2'],
        'model__solver': ['liblinear']
    },
    'RandomForest': {
        'model__n_estimators': randint(50, 200),
        'model__max_depth': randint(2, 20),
        'model__min_samples_split': uniform(0.01, 0.2),
        'model__min_samples_leaf': uniform(0.01, 0.2)
    },
    'GradientBoosting': {
        'model__n_estimators': randint(50, 200),
        'model__learning_rate': uniform(0.01, 0.3),
        'model__max_depth': randint(2, 10),
        'model__min_samples_split': uniform(0.01, 0.2),
        'model__min_samples_leaf': uniform(0.01, 0.2),
        'model__subsample': uniform(0.5, 0.5)
    }
}
models = {
    'LogisticRegression': LogisticRegression(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

### Definição da validação cruzada randomizada

In [9]:
# RandomizedSearchCV para cada modelo
best_model = None
best_score = 0
for model_name, model in models.items():
    pipeline.set_params(model=model)
    search = RandomizedSearchCV(pipeline, param_distributions=param_distributions[model_name], 
                                n_iter=10, cv=10, n_jobs=-1, random_state=42, verbose=0)
    search.fit(X_train, y_train)
    
    # Selecionando o melhor modelo baseado na validação
    score = accuracy_score(y_val, search.best_estimator_.predict(X_val))
    if score > best_score:
        best_score = score
        best_model = search.best_estimator_

### Avaliação final do modelo no conjunto de teste

In [10]:
# Avaliando a performance do melhor modelo no conjunto de teste
test_score = accuracy_score(y_test_final, best_model.predict(X_test_final))
print("Melhor modelo:", best_model)
print("Acurácia no conjunto de teste:", test_score)

Melhor modelo: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['doors', 'persons']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['buying', 'maint',
                                                   'lug_boot', 'safety'])])),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.12562495076197483,
                                            min_samples_leaf=0.02953442280127678,
                                            min_samples_split=0.1468466053024314,
                                            n_estimators=160,
                                            subsample=0.8416317594127292))])
Acurácia no conjunto de teste: 0.9219653179190751
